In [1]:
import os 

import openai 

In [20]:
openai.api_key = os.getenv('OPENAI_API_KEY') 

In [3]:
def create_test_prompt(topic, num_questions, num_possible_answers):
    prompt = f"Create a multiple choice quiz on the topic of {topic} consisting of {num_questions}. "\
            +f"Each question should have {num_possible_answers} options. "\
            +f"Also include the correct answer for each question using the starting string 'Correct Answer:' "
    return prompt

In [4]:
response = openai.Completion.create(engine='text-davinci-003',
                                   prompt=create_test_prompt('US History',4,4),
                                   max_tokens=256,
                                    temperature=0.7)

In [5]:
print(response['choices'][0]['text'])



Q1: Who was the first president of the United States?
A. John Adams
B. Thomas Jefferson
C. George Washington
D. Benjamin Franklin
Correct Answer: C. George Washington

Q2: What year was the Declaration of Independence signed?
A. 1776
B. 1783
C. 1787
D. 1789
Correct Answer: A. 1776

Q3: What event marked the end of the American Revolutionary War?
A. Battle of Bunker Hill
B. Battle of Yorktown
C. Battle of Saratoga
D. Battle of Valley Forge
Correct Answer: B. Battle of Yorktown

Q4: Who wrote the Federalist Papers?
A. Thomas Jefferson
B. Alexander Hamilton
C. James Madison
D. John Adams
Correct Answer: C. James Madison


In [6]:
def create_student_view(test,num_questions):
    student_view = {1:''}
    question_number = 1
    for line in test.split("\n"):
        if not line.startswith("Correct Answer:"):
            student_view[question_number] += line+'\n'
        else:
            if question_number < num_questions:
                question_number +=1
                student_view[question_number] = ''
    return student_view

In [7]:
result = create_student_view(response['choices'][0]['text'],4)

In [8]:
for key in result:
    print(result[key])



Q1: Who was the first president of the United States?
A. John Adams
B. Thomas Jefferson
C. George Washington
D. Benjamin Franklin


Q2: What year was the Declaration of Independence signed?
A. 1776
B. 1783
C. 1787
D. 1789


Q3: What event marked the end of the American Revolutionary War?
A. Battle of Bunker Hill
B. Battle of Yorktown
C. Battle of Saratoga
D. Battle of Valley Forge


Q4: Who wrote the Federalist Papers?
A. Thomas Jefferson
B. Alexander Hamilton
C. James Madison
D. John Adams



In [9]:
def extract_answer(test,num_questions):
    answers = {1:''}
    question_number = 1
    for line in test.split("\n"):
        if line.startswith("Correct Answer:"):
            answers[question_number] += line+'\n'

            if question_number < num_questions:
                question_number +=1
                answers[question_number] = ''
    return answers

In [10]:
extract_answer(response['choices'][0]['text'],4)

{1: 'Correct Answer: C. George Washington\n',
 2: 'Correct Answer: A. 1776\n',
 3: 'Correct Answer: B. Battle of Yorktown\n',
 4: 'Correct Answer: C. James Madison\n'}

In [11]:
student_view = create_student_view(response['choices'][0]['text'],4)

In [12]:
answers = extract_answer(response['choices'][0]['text'],4)

In [13]:
# take the exam
def take(student_view):
    student_answers= {}
    for question, question_view in student_view.items():
        print(question_view)
        answer = input("Enter your answer: ")
        student_answers[question] = answer
    return student_answers

In [14]:
student_answers = take(student_view)



Q1: Who was the first president of the United States?
A. John Adams
B. Thomas Jefferson
C. George Washington
D. Benjamin Franklin

Enter your answer: josh t

Q2: What year was the Declaration of Independence signed?
A. 1776
B. 1783
C. 1787
D. 1789

Enter your answer: 1776

Q3: What event marked the end of the American Revolutionary War?
A. Battle of Bunker Hill
B. Battle of Yorktown
C. Battle of Saratoga
D. Battle of Valley Forge

Enter your answer: battle

Q4: Who wrote the Federalist Papers?
A. Thomas Jefferson
B. Alexander Hamilton
C. James Madison
D. John Adams

Enter your answer: john adams


In [15]:
student_answers

{1: 'josh t', 2: '1776', 3: 'battle', 4: 'john adams'}

In [16]:
answers

{1: 'Correct Answer: C. George Washington\n',
 2: 'Correct Answer: A. 1776\n',
 3: 'Correct Answer: B. Battle of Yorktown\n',
 4: 'Correct Answer: C. James Madison\n'}

In [17]:
def grade(correct_answer_dict, student_answers):
    correct_answers = 0
    for question, answer in student_answers.items():
        if answer.upper() == correct_answer_dict[question][16]:
            correct_answers += 1
    grade = 100*correct_answers / len(answers)
    
    if grade <60:
        passed = "NO PASS"
    else:
        passed = "PASS!"
        
    return f"{correct_answers}/{len(answers)} correct! You got {grade}% grade, {passed}"

In [18]:
grade(answers, student_answers)

'0/4 correct! You got 0.0% grade, NO PASS'